<a href="https://colab.research.google.com/github/cmhrabi/AgenticDesignPatterns/blob/main/notebooks/Chapter_1_Prompt_Chaining_(Code_Example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-community langchain-openai langgraph

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9 MB/s  0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.1 MB/s  0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.8 MB/s  0:00:00
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# For better security, load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv('/content/.env')
# Make sure your OPENAI_API_KEY is set in the .env file

# Initialize the Language Model (using ChatOpenAI is recommended)
llm = ChatOpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
    model="anthropic/claude-sonnet-4",
    temperature=0
    )

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
    "Extract the technical specifications from the following text:\n\n{text_input}"
)

# -- Promp 3: Create a performace rating based on the specifications
prompt_rating = ChatPromptTemplate.from_template(
    "Give a performance rating based on the specification of the computer: \n\n{specs}"
)

# -- Prompt 4: Transform back to JSON
propmt_transform_rating = ChatPromptTemplate.from_template(
    "Transform into JSON object with original specs and performance score with 'cpu', 'memory', 'storage', and 'performance_score' (score from 1-10) as keys: \n\n{performance_rating}"
)

# --- Build the Chain using LCEL ---
# The StrOutputParser() converts the LLM's message output to a simple string.
extraction_chain = prompt_extract | llm | StrOutputParser()


rating_chain = (
    {"specs": extraction_chain}
    | prompt_rating
    | llm
    | StrOutputParser()
)

full_chain = (
    {"performance_rating": rating_chain}
    | propmt_transform_rating
    | llm
    | JsonOutputParser()
)

# --- Run the Chain ---
input_text = "The new laptop model features a 4.5 GHz octa-core processor, 32GB of RAM, and a 2TB NVMe SSD."

# Execute the chain with the input text dictionary.
final_result = full_chain.invoke({"text_input": input_text})

print("\n-- Final JSON Output --")
print(final_result)

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:27: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1



-- Final JSON Output --
{'cpu': '4.5 GHz octa-core', 'memory': '32GB RAM', 'storage': '2TB NVMe SSD', 'performance_score': 8.5}
